In [20]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time
from math import pi
center_x=0
center_y=0

def hole(img):
    img = img[0:500, 100:600]
    img_thresholded = cv2.inRange(img, (20, 20, 0), (200, 200, 100))
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(img_thresholded, cv2.MORPH_OPEN, kernel)
    opening=cv2.bitwise_not(opening)
    im2, contours, hierarchy=cv2.findContours(opening.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    if ( contours and hierarchy.any ):
        i=0
        while i<len(contours):
            if ( hierarchy[0][i][2] != -1 ):
                return True
            i=i+1
    return False

def burn(img):
    #use with channel b
    img = img[0:500, 100:600]
    ret,thresh1 = cv2.threshold(img,5,255,cv2.THRESH_BINARY)
    hist = cv2.calcHist([thresh1],[0],None,[256],[0,256])
    if hist[0][0] > 600:
        return True
    else:
        return False

def deform(img):
    ret,thresh1 = cv2.threshold(img,40,255,cv2.THRESH_BINARY_INV)
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)

    hist = cv2.calcHist([closing],[0],None,[256],[0,256])
    cakez=hist[0][0] #cake area

    im2, contours, hierarchy=cv2.findContours(closing.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    rad=[] 
    for cnt in contours:
        (cx,cy),radius = cv2.minEnclosingCircle(cnt) # crcle radius
        rad.append(radius)
    hierarchy
    rad=np.sort(rad)
    radc=rad[-2] #get the biggest circle(not including frame)
    area=pi * (radc**2)
    difz=cakez*100/area
    if difz< 88 :
        return True
    else:
        return False
    
def broken(img):
    img = img[0:500, 100:600]
    ret,thresh1 = cv2.threshold(img,40,255,cv2.THRESH_BINARY_INV)
    params = cv2.SimpleBlobDetector_Params()

    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)


    # Change thresholds
    params.minThreshold = 100
    params.maxThreshold = 255
    # Filter by Area.
    params.filterByArea = True
    params.minArea = 1000
    params.maxArea=45000
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.8
    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.8
    #Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity=0.99
    # Detect blobs.
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(closing)

    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    im_with_keypoints = cv2.drawKeypoints(closing, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    #cv2.imshow("Keypoints", im_with_keypoints)
    #cv2.waitKey(0)
    if keypoints:
        global center_x
        global center_y
        center_x=keypoints[0].pt[0]
        center_y=keypoints[0].pt[1]
        return False
    else:
        return True
    
def raw(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(image,245,255,cv2.THRESH_BINARY)
    hist = cv2.calcHist([thresh1],[0],None,[256],[0,256])
    if hist[255][0]>2500: 
        return True
    else:
        return False
    
#######################
def check(filename):
    fname=os.path.basename(filename)
    img=cv2.imread(filename, 1)
    start_time = time.time()
    b,g,r = cv2.split(img)
    
    if hole(img):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "SUCH HOLE!!!", (100, 100), font, 3,(255,255,255),2,cv2.LINE_AA)
        cv2.imwrite(os.path.join('./reviewed', fname), img)
        cv2.destroyAllWindows()
        print("hole")
        print("--- %s seconds ---" % (time.time() - start_time))
        return

    if deform(r): #should be before broken
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "MANY DEFORM!!!", (50, 100), font, 3,(255,255,255),2,cv2.LINE_AA)
        cv2.imwrite(os.path.join('./reviewed', fname), img)
        cv2.destroyAllWindows()
        print("deformed")
        print("--- %s seconds ---" % (time.time() - start_time))
        return

    if raw(img):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "VERY RAW!!!", (100, 100), font, 3,(255,255,255),2,cv2.LINE_AA)
        cv2.imwrite(os.path.join('./reviewed', fname), img)
        cv2.destroyAllWindows()
        print("raw")
        print("--- %s seconds ---" % (time.time() - start_time))
        return
    
    if broken(r) is True: #before burned, coz they can be burned too
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "MUCH BROKEN!!!", (30, 100), font, 3,(255,255,255),2,cv2.LINE_AA)
        cv2.imwrite(os.path.join('./reviewed', fname), img)
        cv2.destroyAllWindows()
        print("broken")
        print("--- %s seconds ---" % (time.time() - start_time))
        return
    
    if burn(b):
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "SO FLAME!!!", (100, 100), font, 3,(255,255,255),2,cv2.LINE_AA)
        cv2.imwrite(os.path.join('./reviewed', fname), img)
        cv2.destroyAllWindows()
        print("burned")
        print("--- %s seconds ---" % (time.time() - start_time))
        return
    
    
    #all checks passed
    font = cv2.FONT_HERSHEY_SIMPLEX
    text="x=%i, y=%i"%(center_x,center_y)
    cv2.putText(img, text, (100, 100), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.imwrite(os.path.join('./reviewed', fname), img)
    cv2.destroyAllWindows()
    print("well done")
    print("--- %s seconds ---" % (time.time() - start_time))
    
############################################################
############################################################
#                                                          #
#######Uncomment path to proceed the whole directory########
#                                                          #
############################################################
############################################################

#mypath=('.\Tortas Unitarias\Tortas buenas')
#mypath = ('.\Tortas Unitarias\Tortas malas\Agujereadas')
#mypath =('.\Tortas Unitarias\Tortas malas\Rotas')
#mypath =('.\Tortas Unitarias\Tortas malas\Quemadas')
#mypath = ('.\Tortas Unitarias\Tortas malas\Deformadas')
#mypath = ('.\Tortas Unitarias\Tortas malas\Crudas')
mypath = ('.\Tortas Unitarias\Tortas indeterminadas')
f = []
for (dirpath, dirnames, filenames) in os.walk(mypath):
    f.extend(filenames)
    break
for f1 in f:
    file=mypath+'\\'+f1 
    print (file)
    check(file)



.\Tortas Unitarias\Tortas indeterminadas\I1.BMP
hole
--- 0.018561363220214844 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I10.BMP
broken
--- 0.03309488296508789 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I11.BMP
broken
--- 0.0589909553527832 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I12.BMP
well done
--- 0.0342411994934082 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I2.BMP
deformed
--- 0.021009445190429688 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I3.BMP
deformed
--- 0.012295961380004883 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I4.BMP
broken
--- 0.05819225311279297 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I5.BMP
hole
--- 0.022027969360351562 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I6.BMP
well done
--- 0.048874616622924805 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I7.BMP
broken
--- 0.03527045249938965 seconds ---
.\Tortas Unitarias\Tortas indeterminadas\I8.BMP
burned
--- 0.03367304801940918

In [50]:
#deformed
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Deformadas\D1.BMP',1)
#img = cv2.imread('.\Tortas Unitarias\Tortas indeterminadas\I3.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Rotas\R2.BMP',1)

imb = cv2.imread('.\Tortas Unitarias\Tortas buenas\B4.BMP', 1)
b,g,r = cv2.split(img)
fname='t1.jpg'
ret,thresh1 = cv2.threshold(r,40,255,cv2.THRESH_BINARY_INV)
kernel = np.ones((50,50),np.uint8)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
dilation = cv2.dilate(thresh1,kernel,iterations = 10)
erosion = cv2.erode(thresh1,kernel,iterations = 3)
opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)


params = cv2.SimpleBlobDetector_Params()

# Detect blobs.
# Change thresholds
params.minThreshold = 100
params.maxThreshold = 255
# Filter by Area.
params.filterByArea = True
params.minArea = 1000
params.maxArea=45000
# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.87
# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.94
#params.filterByConvexity = True
#params.minConvexity=0
detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(thresh1)
 
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(thresh1, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show keypoints
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)
if keypoints:
    print ("True")
#cv2.imshow('image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [57]:
#burned
import numpy as np
import cv2
import os
import time
from matplotlib import pyplot as plt
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Quemadas\Q1.BMP',1)
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Agujereadas\A1.BMP',1)

imb = cv2.imread('.\Tortas Unitarias\Tortas buenas\B7.BMP', 1)
b,g,r = cv2.split(imb)

start_time = time.time()
ret,thresh1 = cv2.threshold(b,40,255,cv2.THRESH_BINARY)
kernel = np.ones((2,2),np.uint8)
dilation = cv2.dilate(thresh1,kernel,iterations = 4)
im2, contours, hierarchy=cv2.findContours(thresh1.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
if ( contours and hierarchy.any ):
    i=0
    while i<len(contours):
        if ( hierarchy[0][i][2] != -1 ):
            print("AAA")
        i=i+1

print("--- %s seconds ---" % (time.time() - start_time))
cv2.imshow('image', thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

AAA
--- 0.00250244140625 seconds ---


In [34]:
#broken
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Rotas\R4.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas buenas\B1.BMP', 1)
img=cv2.imread('.\Tortas Unitarias\Tortas malas\Quemadas\Q2.bmp',1)
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Deformadas\D1.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Crudas\C2.BMP',1)

b,g,r = cv2.split(img)

img = img[0:500, 100:600]
ret,thresh1 = cv2.threshold(r,40,255,cv2.THRESH_BINARY_INV)
params = cv2.SimpleBlobDetector_Params()

kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)


# Change thresholds
params.minThreshold = 100
params.maxThreshold = 255
# Filter by Area.
params.filterByArea = True
params.minArea = 1000
params.maxArea=45000
# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.8
# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.8
#Filter by Convexity
params.filterByConvexity = True
params.minConvexity=0.986
# Detect blobs.
detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(closing)
 
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(closing, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show keypoints
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)
if keypoints:
    print ("True")

#cv2.imshow('image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

True


In [13]:
#raw
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Crudas\C2.BMP',1)
#imb = cv2.imread('.\Tortas Unitarias\Tortas buenas\B1.BMP', 1)

image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
r=image

ret,thresh1 = cv2.threshold(r,245,255,cv2.THRESH_BINARY)

hist = cv2.calcHist([thresh1],[0],None,[256],[0,256])
if hist[255][0]>2500:
    print(hist[255][0])
cv2.imshow('image',thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

6626.0


In [34]:
#burned2
#burned
import numpy as np
import cv2
import os
import time
from matplotlib import pyplot as plt
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Quemadas\Q3.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Rotas\R2.BMP',1)
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Agujereadas\A1.BMP',1)

imb = cv2.imread('.\Tortas Unitarias\Tortas buenas\B8.BMP', 1)

b,g,r = cv2.split(img)
b = b[0:500, 100:600]
ret,thresh1 = cv2.threshold(b,5,255,cv2.THRESH_BINARY)
hist = cv2.calcHist([thresh1],[0],None,[256],[0,256])


cv2.imshow('image', thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

hist[0][0]


2297.0

In [100]:
#remove background
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Quemadas\Q1.BMP',1)
imb = cv2.imread('.\Tortas Unitarias\Tortas buenas\B1.BMP', 1)
#b,g,r = cv2.split(img)

tmp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,alpha = cv2.threshold(tmp,90,255,cv2.THRESH_BINARY_INV)
b, g, r = cv2.split(img)
rgba = [b,g,r, alpha]
dst = cv2.multiply(1-alpha,tmp)
dst1=cv2.add(alpha,tmp)

cv2.imshow('image', dst1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [135]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Agujereadas\A3.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Rotas\R2.BMP',1)

b,g,r = cv2.split(img)
r = r[0:500, 100:600]
ret,thresh1 = cv2.threshold(img,40,255,cv2.THRESH_BINARY)
kernel = np.ones((2,2),np.uint8)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
dilation = cv2.dilate(thresh1,kernel,iterations = 3)
erosion = cv2.erode(thresh1,kernel,iterations = 1)
im2, contours, hierarchy=cv2.findContours(thresh1.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
if ( contours and hierarchy.any ):
    i=0
    while i<len(contours):
        if ( hierarchy[0][i][2] != -1 ):
            print("AAA")
        i=i+1

cv2.imshow('image', thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()
hierarchy
#contours

error: D:\Build\OpenCV\opencv-3.3.1\modules\imgproc\src\contours.cpp:199: error: (-210) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function cvStartFindContours_Impl


In [12]:
#hole
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Agujereadas\A4.BMP',1)
#img = cv2.imread('.\Tortas Unitarias\Tortas malas\Rotas\R1.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas buenas\B1.BMP', 1)

img = img[0:500, 100:600]
img_thresholded = cv2.inRange(img, (20, 20, 0), (200, 200, 100))
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(img_thresholded, cv2.MORPH_OPEN, kernel)
dilation = cv2.dilate(img_thresholded,kernel,iterations = 1)
erosion = cv2.erode(img_thresholded,kernel,iterations = 2)
opening=cv2.bitwise_not(opening)
im2, contours, hierarchy=cv2.findContours(opening.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
cv2.imshow('image', opening)
cv2.waitKey(0)
cv2.destroyAllWindows()
hierarchy

array([[[-1, -1, -1, -1]]], dtype=int32)

In [46]:
#deformed2

#minEnclosingCircle

#deformed
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from math import pi as pi
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Deformadas\D1.BMP',1)
#img = cv2.imread('.\Tortas Unitarias\Tortas indeterminadas\I3.BMP',1)
img = cv2.imread('.\Tortas Unitarias\Tortas malas\Rotas\R2.BMP',1)

imb = cv2.imread('.\Tortas Unitarias\Tortas buenas\B4.BMP', 1)
b,g,r = cv2.split(img)
ret,thresh1 = cv2.threshold(r,40,255,cv2.THRESH_BINARY_INV)
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)

hist = cv2.calcHist([closing],[0],None,[256],[0,256])
cakez=hist[0][0]

im2, contours, hierarchy=cv2.findContours(closing.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
rad=[] 
for cnt in contours:
    (cx,cy),radius = cv2.minEnclosingCircle(cnt)
    rad.append(radius)
hierarchy
rad=np.sort(rad)
radc=rad[-2]
area=pi * (radc**2)

difz=cakez*100/area
if difz< 88 :
    print("deformed!!!")
print("cake",hist[0][0])
print("cicle",area)
print("difz",difz)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


cake 39428.0
cicle 43920.4083211
difz 89.771478698


In [ ]:
#deformed prod(bad
    ret,thresh1 = cv2.threshold(img,40,255,cv2.THRESH_BINARY_INV)
    params = cv2.SimpleBlobDetector_Params()
    # Change thresholds
    params.minThreshold = 0
    params.maxThreshold = 255
    # Filter by Area.
    params.filterByArea = True
    params.minArea = 1000
    params.maxArea=45000
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.81
    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.9
    # Detect blobs.
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(thresh1)

In [21]:
#find all centers
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time

def check (filename):
    fname=os.path.basename(filename)
    start_time = time.time()
    
    img = cv2.imread(file,1)
    b,g,r=cv2.split(img)

    ret,thresh1 = cv2.threshold(r,40,255,cv2.THRESH_BINARY_INV)
    params = cv2.SimpleBlobDetector_Params()

    kernel = np.ones((10,10),np.uint8)
    closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)

    # Change thresholds
    params.minThreshold = 0
    params.maxThreshold = 255
    # Filter by Area.
    params.filterByArea = True
    params.minArea = 1000
    params.maxArea=45000
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.5
    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.5
    #Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity=0.5
    # Detect blobs.
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(closing)

    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
    im_with_keypoints = cv2.drawKeypoints(closing, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    #cv2.imshow("Keypoints", im_with_keypoints)
    #cv2.waitKey(0)
    split=0
    text_f=""
    font = cv2.FONT_HERSHEY_SIMPLEX
    for center in keypoints:
        center_x=center.pt[0]
        center_y=center.pt[1]
        text="x=%i, y=%i"%(center_x,center_y)
        y=100+split
        cv2.putText(img, text, (100, y), font, 1,(255,255,255),2,cv2.LINE_AA)
        split=split+50


    cv2.imwrite(os.path.join('./reviewed_amp', fname), img)
    cv2.destroyAllWindows()
    #cv2.imshow("Keypoints", img)
    #cv2.waitKey(0)
    print("done")
    print("--- %s seconds ---" % (time.time() - start_time))
    
    
mypath=('.\Tortas Ampliacion')
f = []
for (dirpath, dirnames, filenames) in os.walk(mypath):
    f.extend(filenames)
    break
for f1 in f:
    file=mypath+'\\'+f1 
    print (file)
    check(file)

.\Tortas Ampliacion\Imagen06-22-2016 - 17.11.25.469.BMP
done
--- 0.05982232093811035 seconds ---
.\Tortas Ampliacion\Imagen06-22-2016 - 17.11.47.569.BMP
done
--- 0.045973777770996094 seconds ---
.\Tortas Ampliacion\Imagen06-23-2016 - 14.31.41.956.BMP
done
--- 0.06799697875976562 seconds ---
.\Tortas Ampliacion\Imagen06-23-2016 - 14.33.12.250.BMP
done
--- 0.08765363693237305 seconds ---
.\Tortas Ampliacion\Imagen06-23-2016 - 14.36.33.505.BMP
done
--- 0.06400179862976074 seconds ---
